### Imports

In [2]:
# The code below uses few-shot learning to generalize a fine-tuned model on another task.

import os
import torch
from transformers import AutoImageProcessor, SwinForImageClassification, TrainingArguments, Trainer
import evaluate
from datasets import load_dataset, DatasetDict
import numpy as np
import random
from torch.utils.data import Dataset
from torchinfo import summary

print(os.getcwd())
os.chdir("..") # have to go up one directory, can also use os.chdir("..")
print(os.getcwd())

# CUDA check 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

c:\Users\metet\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\metet\OneDrive\Documents\GitHub\thesis\RQ3
c:\Users\metet\OneDrive\Documents\GitHub\thesis
cuda


### Define Model and Data

In [43]:
# Choose model
# model_path = './sdxl-fine-tune'
# model_path = './sdxl-fine-tune-art'
# model_path = './sdxl-fine-tune_few_shot'
model_path = './sdxl-fine-tune-art_few_shot'

processor = AutoImageProcessor.from_pretrained(model_path)
#classifier = pipeline("image-classification", model=model_path, device=0 if torch.cuda.is_available() else -1)

# Choose dataset
# dataset_path = 'archive/datasets/faces_512x512'
# dataset_path = 'archive/datasets/art_512x512'
dataset_path = 'archive/datasets/dogs'

ds = load_dataset("imagefolder", data_dir=dataset_path)
print(ds)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 50
    })
})


### Custom Dataset Class 

In [44]:
# for the life of me i couldn't get the code to work with the normal huggingface dataset (despite the
# fact that the logic IS IDENTICAL to rq2_finetuning, but i digress) so here's a custom dataset class)

class CustomDataset(Dataset):
    def __init__(self, hf_dataset, processor, device):
        self.dataset = hf_dataset
        self.processor = processor
        self.device = device
        
    def __len__(self):
        return len(self.dataset)
    
    # manaully processes each image so pixel values are always returned ('image' is no longer needed for Trainer)
    def __getitem__(self, idx):
        item = self.dataset[idx]
        inputs = self.processor(images=item['image'], return_tensors="pt")
        return {
            'pixel_values': inputs['pixel_values'].squeeze(0),
            'labels': torch.tensor(item['label'])
        }

### Pre-Processing

In [45]:
# Turn dicts into tensors
def collate_fn(batch):
    pixel_values = torch.stack([x['pixel_values'] for x in batch])
    labels = torch.tensor([x['labels'] for x in batch])
    return {
        'pixel_values': pixel_values.to(device),
        'labels': labels.to(device)
    }

In [46]:
# Define metrics
acc_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

def compute_metrics(p):
    acc = acc_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)
    f1 = f1_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)
    return {"Accuracy": acc["accuracy"], "F1": f1["f1"]}

### Create Few-Shot Set

In [47]:
def create_few_shot_set(dataset, set_size, seed): # set_size = 10 -> 5-shot learning (2 classes)
    random.seed(seed)

    newset_indices = []

    label0_indices = [i for i, label in enumerate(dataset['train']['label']) if label == 0]
    label1_indices = [i for i, label in enumerate(dataset['train']['label']) if label == 1]

    # get equal number of samples from each class
    newset0_indices = random.sample(label0_indices, set_size // 2)
    newset1_indices = random.sample(label1_indices, set_size // 2)
    newset_indices = newset0_indices + newset1_indices

    random.shuffle(newset_indices)

    few_shot_train = dataset['train'].select(newset_indices)
    val_set = dataset['validation'] # can use the full validation set

    return DatasetDict({
        'train': few_shot_train,
        'validation': val_set
    })

print(type(ds))

few_shot_ds = create_few_shot_set(ds, set_size=20, seed=44)

print(type(few_shot_ds))
print(few_shot_ds)

train_dataset = CustomDataset(few_shot_ds['train'], processor, device)
val_dataset = CustomDataset(few_shot_ds['validation'], processor, device)

# sample = train_dataset[0]
# print("Pixel values shape:", sample['pixel_values'].shape)
# print("Label:", sample['labels'])

<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 100
    })
})


### Load Model

In [48]:
# Use current dataset to extract the labels (doesn't really matter which dataset we use)
labels = ds['train'].features['label'].names
print(labels[0:2])

# Load the pre-trained model
model = SwinForImageClassification.from_pretrained(
    model_path, 
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(device)

['0', '1']


### Prepare Model For Training

In [49]:
# Freezing earlier layers
print(summary(model, input_size=(1, 3, 224, 224)))

for param in model.parameters():
    param.requires_grad = False # freeze all layers

for param in model.classifier.parameters():
    param.requires_grad = True # unfreeze the classifier layer

for param in model.swin.encoder.layers[-1].parameters():
    param.requires_grad = True # unfreeze the last layer of the encoder

for name, param in model.named_parameters():
    if "layernorm" in name.lower():
        param.requires_grad = True # unfreeze layernorm layers 


Layer (type:depth-idx)                                            Output Shape              Param #
SwinForImageClassification                                        [1, 2]                    --
├─SwinModel: 1-1                                                  [1, 1024]                 --
│    └─SwinEmbeddings: 2-1                                        [1, 3136, 128]            --
│    │    └─SwinPatchEmbeddings: 3-1                              [1, 3136, 128]            6,272
│    │    └─LayerNorm: 3-2                                        [1, 3136, 128]            256
│    │    └─Dropout: 3-3                                          [1, 3136, 128]            --
│    └─SwinEncoder: 2-2                                           [1, 49, 1024]             --
│    │    └─ModuleList: 3-4                                       --                        86,734,648
│    └─LayerNorm: 2-3                                             [1, 49, 1024]             2,048
│    └─AdaptiveAvgPool1d: 2-4 

In [50]:
# Training arguments
training_args = TrainingArguments(
    output_dir=model_path + "_few_shot",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=5, # increase num of epochs for longer training with smaller LR
    learning_rate=1e-5, # need low learning rate to avoid forgetting of original dataset stuff
    weight_decay=1e-4, # adds regularization to all layers, can help generalize better (?)
    logging_dir="./logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none",
    dataloader_pin_memory=False, # causes runtime error otherwise
)

In [51]:
from transformers import EarlyStoppingCallback

# Trainer 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=15)],
)

C:\Users\metet\AppData\Local\Temp\ipykernel_11540\1595343547.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Training and Evaluation

In [52]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

metrics = trainer.evaluate(val_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.495385,0.780000,0.800000
2,0.565200,0.468643,0.800000,0.814815
3,0.565200,0.450751,0.800000,0.814815
4,0.470300,0.440847,0.800000,0.814815
5,0.470300,0.437366,0.800000,0.814815


c:\Users\metet\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 128}
  warnings.warn(
c:\Users\metet\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will bec

***** train metrics *****
  epoch                    =        5.0
  total_flos               =  7296498GF
  train_loss               =     0.5065
  train_runtime            = 0:00:13.13
  train_samples_per_second =      7.611
  train_steps_per_second   =      1.903


***** eval metrics *****
  epoch                   =        5.0
  eval_Accuracy           =        0.8
  eval_F1                 =     0.8148
  eval_loss               =     0.4374
  eval_runtime            = 0:00:01.49
  eval_samples_per_second =       67.0
  eval_steps_per_second   =       8.71
